## importing_libs

In [35]:
import tensorflow as tf
import tensorflow.keras.backend

config = tf.compat.v1.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 0.6 sometimes works better for folks
config.gpu_options.allow_growth = True

tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

In [32]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

AttributeError: module 'tensorflow.keras.backend' has no attribute 'set_session'

In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
MIXED_PRECISION = True
XLA_ACCELERATE  = False # Didn't work; Dunno Why!

GPUS = tf.config.experimental.list_physical_devices('GPU')
if GPUS:
    try:
        for GPU in GPUS:
            tf.config.experimental.set_memory_growth(GPU, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(GPUS), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
    except RuntimeError as  RE:
        print(RE)

if MIXED_PRECISION:
    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    tf.keras.mixed_precision.experimental.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')
    
strategy = tf.distribute.get_strategy()
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}') 
print("Tensorflow version " + tf.__version__)

1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce GTX 1650, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Mixed precision enabled
REPLICAS: 1
Tensorflow version 2.5.0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [4]:
print(tf.version.VERSION)

2.5.0


## importing_data

In [5]:
csv_file = "C://Users//sdkay//Documents//INPT//2A//s4//PFA//prj_tech//data//fer2013.csv"

df = pd.read_csv(csv_file)

In [6]:
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [7]:
df.isna().sum()

emotion    0
pixels     0
Usage      0
dtype: int64

In [8]:
df.dtypes

emotion     int64
pixels     object
Usage      object
dtype: object

In [9]:
df.Usage.unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

## raw_train_set

In [10]:
df_train = df[df["Usage"] == "Training"]

In [11]:
raw_X_train = df_train["pixels"].to_list()
raw_y_train = df_train["emotion"].to_list()

In [12]:
len(raw_X_train)

28709

In [13]:
len(raw_X_train[0].split())**0.5

48.0

In [14]:
len(raw_y_train)

28709

## raw_test_set

In [15]:
df_test = df[df["Usage"] != "Training"]

In [16]:
raw_X_test = df_test["pixels"].to_list()
raw_y_test = df_test["emotion"].to_list()

In [17]:
len(raw_X_test)

7178

In [18]:
len(raw_y_test)

7178

## train_set

In [19]:
X_train = []
y_train = np.array(raw_y_train)

for e in raw_X_train : 
    X_train.append(list(map(int, e.split())))

X_train = np.array(X_train).reshape((28709 , 48 , 48 , 1))

## test_set

In [20]:
X_test = []
y_test = np.array(raw_y_test)

for e in raw_X_test : 
        X_test.append(list(map(int, e.split())))

X_test = np.array(X_test).reshape((7178 , 48 , 48 , 1))
    

In [21]:
classes = ['Angry' , 'Disgust' , 'Fear' , 'Happy' , 'Sad' , 'Surprise' , 'Neutral'] 

## normalisation

In [22]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [23]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(
                                                                                                            filters = 32, kernel_size = (3,3) , activation = 'relu' , input_shape = (48,48,1)),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Conv2D(filters = 64,kernel_size = (3,3) , activation = 'relu' , input_shape = (48,48,1)),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3) , activation = 'relu'),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3) , activation = 'relu'),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Flatten(),

                                                                                                            tf.keras.layers.Dense(100 , activation = 'relu'),
                                                                                                            tf.keras.layers.Dense(7 , activation = 'softmax')                          
                                                                                                            ])

In [24]:
# Create the model
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001),input_shape=(48,48,1)))
model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(tf.keras.layers.Conv2D(7, kernel_size=(1, 1), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
# model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(7, kernel_size=(4, 4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Activation("softmax"))

model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 32)        128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 20, 128)      

In [34]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet' , include_top=False , input_shape = (48, 48, 3) )

In [35]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [38]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/Users/sdkay/Documents/INPT/2A/s4/PFA/prj_tech/data/FER2013'
train_dir = os.path.join(base_dir, 'train')
#validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 1, 1, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory , target_size=(48, 48) , batch_size=batch_size , class_mode='categorical')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
#validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 28709 images belonging to 7 classes.


ValueError: could not broadcast input array from shape (20,7) into shape (20)

In [25]:
model.compile(
                          optimizer = tf.keras.optimizers.RMSprop(learning_rate = .0001),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=100 , validation_split = .1 )

Epoch 1/100
808/808 [==============================] - 232s 249ms/step - loss: 1.8368 - accuracy: 0.2512 - val_loss: 1.7453 - val_accuracy: 0.2605
Epoch 2/100
808/808 [==============================] - 198s 245ms/step - loss: 1.6712 - accuracy: 0.3396 - val_loss: 1.5982 - val_accuracy: 0.3821
Epoch 3/100
808/808 [==============================] - 255s 315ms/step - loss: 1.5503 - accuracy: 0.4135 - val_loss: 1.5174 - val_accuracy: 0.4124
Epoch 4/100
808/808 [==============================] - 247s 306ms/step - loss: 1.4675 - accuracy: 0.4455 - val_loss: 1.4323 - val_accuracy: 0.4525
Epoch 5/100
808/808 [==============================] - 211s 262ms/step - loss: 1.4140 - accuracy: 0.4669 - val_loss: 1.4220 - val_accuracy: 0.4615
Epoch 6/100
808/808 [==============================] - 185s 229ms/step - loss: 1.3633 - accuracy: 0.4886 - val_loss: 1.3943 - val_accuracy: 0.4692
Epoch 7/100
808/808 [==============================] - 187s 232ms/step - loss: 1.3203 - accuracy: 0.5042 - val_loss: 1

KeyboardInterrupt: 

In [ ]:
model.compile(optimizer='rmsprop' , loss='categorical_crossentropy' , metrics=['accuracy'])

model.fit(train_images , train_labels , epochs=5 , batch_size=64)

In [ ]:
string_list = ["1", "2", "3"]

integer_map = map(int, string_list)
integer_list = list(integer_map)
print(integer_list)

In [ ]:
import pandas as pd

df = pd.DataFrame({'pixels': ['10 20 30 40', '11 12 13 14']})

a = (
    df['pixels'].str.split(' ', expand=True)
        .astype(int).to_numpy()
        .reshape((2, 2, 2, 1))
)
print(a)

In [ ]:
df["pixels"].str.split(expand = True).astype(int).to_numpy()

In [ ]:
a.shape

In [ ]:
df_train['pixels'].str.split(expand = True).astype(int).to_numpy().reshape((28709 , 48 , 48))